---
### Objetivo:

Uma analise comparativa de pandas X spark (de preferencia no databricks)

### O que fazer:

Pegar 2 datasets um > 50mb e outro menor

Realizar pre-processamento de dados e analise exploratoria dos dados, usando pandas e pyspark

Identificar quando o panda ou pyspark é melhor

### Datasets escolhidos:

https://www.kaggle.com/datasets/swatikhedekar/exploratory-data-analysis-on-netflix-data

Dataset público com informações sobre série e filmes disponiveis na plataforma de streaming Netflix de 2008 a 2021.

Tamanho do dataset = **3.4 mb**


https://www.kaggle.com/datasets/peacehegemony/history-of-music-bnb

Dataset com metadata sobre o catalogo de músicas na livraria britânica em http://explore.bl.uk

Tamanho do dataset = **261 mb**

---

In [1]:
# Importando as bibliotecas que serão utilizadas
import time # para registrar o tempo de execução
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import *

---
### Trabalhando com o Dataset menor usando PANDAS
---

In [2]:
inicio_pandas_netflix = time.time()

In [3]:
# Leitura do arquivo csv

netflix = pd.read_csv('netflix_titles_2021.csv')
netflix.head(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [4]:
#verificando o tamanho do dataset

netflix.shape

(8807, 12)

In [5]:
# Alterando os nomes das colunas

colunas = {'type':'tipo', 'title':'titulo', 'director':'diretor', 'cast':'elenco',
            'country':'pais', 'date_added':'adicionado_em','release_year':'ano_lancamento',
            'rating':'avaliacao', 'duration':'duracao', 'listed_in':'categorias', 'description':'descricao'}

netflix.rename(columns=colunas, inplace=True)

In [6]:
netflix.head()

,show_id,tipo,titulo,diretor,elenco,pais,adicionado_em,ano_lancamento,avaliacao,duracao,categorias,descricao
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [7]:
# Contagem percentual de valores nulos no dataframe

frequencia = netflix.isna().sum() 
percentual_na = round((netflix.isna().sum() / len(netflix))*100, 2)
valores_na = pd.DataFrame([frequencia, percentual_na]).transpose().rename(columns={0:'Frequência', 1:'% de NAs'})
valores_na

,Frequência,% de NAs
show_id,0.0,0.00
tipo,0.0,0.00
titulo,0.0,0.00
diretor,2634.0,29.91
elenco,825.0,9.37
pais,831.0,9.44
adicionado_em,10.0,0.11
ano_lancamento,0.0,0.00
avaliacao,4.0,0.05
duracao,3.0,0.03


In [8]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   show_id         8807 non-null   object
 1   tipo            8807 non-null   object
 2   titulo          8807 non-null   object
 3   diretor         6173 non-null   object
 4   elenco          7982 non-null   object
 5   pais            7976 non-null   object
 6   adicionado_em   8797 non-null   object
 7   ano_lancamento  8807 non-null   int64 
 8   avaliacao       8803 non-null   object
 9   duracao         8804 non-null   object
 10  categorias      8807 non-null   object
 11  descricao       8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [9]:
# Otimizando o tamanho da variável ano_lancamento

netflix['ano_lancamento'] = netflix["ano_lancamento"].astype('int16')

In [10]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   show_id         8807 non-null   object
 1   tipo            8807 non-null   object
 2   titulo          8807 non-null   object
 3   diretor         6173 non-null   object
 4   elenco          7982 non-null   object
 5   pais            7976 non-null   object
 6   adicionado_em   8797 non-null   object
 7   ano_lancamento  8807 non-null   int16 
 8   avaliacao       8803 non-null   object
 9   duracao         8804 non-null   object
 10  categorias      8807 non-null   object
 11  descricao       8807 non-null   object
dtypes: int16(1), object(11)
memory usage: 774.2+ KB


In [11]:
# Cria um dicionário com os valores dos meses

import calendar

meses = dict((month, str(index)) for index, month in enumerate(calendar.month_name) if month)
meses

{'January': '1',
 'February': '2',
 'March': '3',
 'April': '4',
 'May': '5',
 'June': '6',
 'July': '7',
 'August': '8',
 'September': '9',
 'October': '10',
 'November': '11',
 'December': '12'}

In [12]:
# modificando o texto para achar um padrão para o split 
netflix['adicionado_em'] = netflix.adicionado_em.str.strip()
netflix['adicionado_em'] = netflix['adicionado_em'].str.replace(', ', '/')
netflix['adicionado_em'] = netflix['adicionado_em'].str.replace(' ', '/')

In [13]:
# separação do valor adicionado_em em dia, mês e ano

data = netflix['adicionado_em'].str.split('/', n = 2, expand = True)
mes = data[0]
dia = data[1]
ano = data[2]

In [14]:
# substituição dos nomes dos meses por seus respectivos valores

mes.replace(meses, inplace=True)

In [15]:
# novo dataframe pandas para inclusão da coluna de data

data = pd.DataFrame([dia, mes, ano]).transpose().rename(columns={1:'dia', 0:'mes', 2:'ano'})
data.dia = data.dia.str.zfill(2)
data.mes = data.mes.str.zfill(2)
data['data'] = data.dia + '/' + data.mes + '/' + data.ano

In [16]:
# remoção das colunas dia, mes e ano
data.drop(columns=['dia', 'mes', 'ano'], inplace=True)

In [17]:
# modifica a coluna adicionado_em para valores com datas no formato dd/mm/yyyy.
netflix['adicionado_em'] = data
netflix.head(5)

,show_id,tipo,titulo,diretor,elenco,pais,adicionado_em,ano_lancamento,avaliacao,duracao,categorias,descricao
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,25/09/2021,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,24/09/2021,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,24/09/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,24/09/2021,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,24/09/2021,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [18]:
netflix['adicionado_em'] = netflix['adicionado_em'].astype('datetime64[ns]')

C:\Users\PC MATRIX\AppData\Local\Temp\ipykernel_984\3237889366.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  netflix['adicionado_em'] = netflix['adicionado_em'].astype('datetime64[ns]')


In [19]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   show_id         8807 non-null   object        
 1   tipo            8807 non-null   object        
 2   titulo          8807 non-null   object        
 3   diretor         6173 non-null   object        
 4   elenco          7982 non-null   object        
 5   pais            7976 non-null   object        
 6   adicionado_em   8797 non-null   datetime64[ns]
 7   ano_lancamento  8807 non-null   int16         
 8   avaliacao       8803 non-null   object        
 9   duracao         8804 non-null   object        
 10  categorias      8807 non-null   object        
 11  descricao       8807 non-null   object        
dtypes: datetime64[ns](1), int16(1), object(10)
memory usage: 774.2+ KB


In [ ]:
# Agrupando os valores por categorias

categorias = netflix['categorias'].value_counts()
categorias

In [ ]:
# Top 3 categorias

top_3_categorias = categorias.head(3)
top_3_categorias


In [ ]:
# função para add valores nas barras 
def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i]//2, y[i], ha = 'center', color = 'white')


plt.bar(x=top_3_categorias.index, height=top_3_categorias)
plt.xlabel('categorias')
plt.title('Top 3 Categorias')
addlabels(x=top_3_categorias.index, y=top_3_categorias)
plt.show()

In [ ]:
# Top 5 Países
paises = netflix.pais.value_counts().head(5)
paises
label = paises.index
myexplode = [0.2, 0, 0, 0, 0]
plt.pie(paises, labels=label, explode=myexplode)
plt.title('Top 5 países')
plt.show()

In [38]:
# retornado os valores nulos do dataset

netflix_transformado = netflix.dropna()
netflix_transformado.head(5)

,show_id,tipo,titulo,diretor,elenco,pais,adicionado_em,ano_lancamento,avaliacao,duracao,categorias,descricao
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,2021-09-24,2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021-09-24,2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",2021-09-23,2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...
24,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,2021-09-21,1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...


In [39]:
netflix_transformado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5332 entries, 7 to 8806
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   show_id         5332 non-null   object        
 1   tipo            5332 non-null   object        
 2   titulo          5332 non-null   object        
 3   diretor         5332 non-null   object        
 4   elenco          5332 non-null   object        
 5   pais            5332 non-null   object        
 6   adicionado_em   5332 non-null   datetime64[ns]
 7   ano_lancamento  5332 non-null   int16         
 8   avaliacao       5332 non-null   object        
 9   duracao         5332 non-null   object        
 10  categorias      5332 non-null   object        
 11  descricao       5332 non-null   object        
dtypes: datetime64[ns](1), int16(1), object(10)
memory usage: 510.3+ KB


In [ ]:
# separando o dataset por duração (Season)

selecao = netflix_transformado.duracao.str.contains('Season')
season = netflix_transformado[selecao]
season.head()

In [ ]:
season.info()

In [ ]:
# separando o dataset por duração (Minutos)

selecao = netflix_transformado.duracao.str.contains('min')
filmes_minutos = netflix_transformado[selecao]
filmes_minutos.reset_index(inplace=True, drop=True)
filmes_minutos.head()

In [ ]:
filmes_minutos.info()

In [ ]:
# tranformando o valor da duração em minutos (str) para int
filmes_minutos['duracao'] = filmes_minutos.loc[:, 'duracao'].str.replace(' min', '')
filmes_minutos.head(5)

In [ ]:
filmes_minutos.duracao = filmes_minutos.loc[:, 'duracao'].astype('int16')

In [ ]:
filmes_minutos.info()

In [ ]:
# Média de duração dos filmes (min)

media_duracao = round(filmes_minutos.duracao.mean(), 2)
media_duracao

In [ ]:
# Verificando as estatísticas básicas do dataset filmes_minutos

filmes_minutos.duracao.describe()

In [ ]:
# boxplot - verificando outliers

bp = plt.boxplot(x='duracao', data=filmes_minutos, vert=1, patch_artist=True, labels=['Duração'])
colors = 'blue'
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
for median in bp['medians']:
    median.set(color ='red',
               linewidth = 3)
for whisker in bp['whiskers']:
    whisker.set(color ='red',
                linewidth = 1.5,
                linestyle =":")
plt.show()

In [20]:
fim_pandas_netflix = time.time()

---
### Trabalhando com o Dataset menor usando SPARK
---

In [21]:
inicio_spark_netflix = time.time()

In [40]:
## iniciando o spark

spark = SparkSession.builder.appName("Netflix Data").getOrCreate()


In [41]:
#lendo o arquivo .csv

netflix_spark = spark.read.csv("netflix_titles_2021.csv", header=True, inferSchema=True)

netflix_spark.show(10)

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglan

In [51]:
#verificando o tamanho do dataset

print((netflix_spark.count(),len(netflix_spark.columns)))

(8809, 12)


show_id           8807
tipo              8807
titulo            8807
diretor           6173
elenco            7982
pais              7976
adicionado_em     8797
ano_lancamento    8807
avaliacao         8803
duracao           8804
categorias        8807
descricao         8807
dtype: int64

In [24]:
fim_spark_netflix = time.time()

---
### Trabalhando com o Dataset maior usando PANDAS
---

In [25]:
inicio_pandas_musica = time.time()

In [26]:
# Leitura do arquivo csv

musicas = pd.read_csv('2detailedrecords.csv', engine='python', on_bad_lines='skip')
musicas.head(10)

,BL record ID,Composer,Composer life dates,Title,Standardised title,Other titles,Other names,Publication date (standardised),Publication date (not standardised),Country of publication,...,Contents,Referenced in,Subject/genre terms,Physical description,Series title,Number within series,ISBN,ISMN,Publisher number,BL shelfmark
0,1279866,NaN,NaN,The Penguin book of Canadian Folk songs,NaN,Canadian folk songs,"Fowke, Edith ; MacMillan, Keith (Keith Campbell)",1973.0,1973,England,...,A Fenian song -- Bold Wolfe -- The battle of t...,NaN,"Folk songs--Canada ; Folk songs, French--Canada","224 pages, music, 20 cm",NaN,NaN,0140708421,NaN,NaN,mH00/3305 ; X.439/3548
1,1312079,NaN,NaN,Anthology for The musician's guide to theory a...,NaN,"Musician's guide, anthology ; Musician's guide...","Clendinning, Jane Piper ; Marvin, Elizabeth West",2005.0,c2005,New York (State),...,NaN,NaN,Musical analysis--Music collections,"1 score (vii, 260 pages), 28 cm",NaN,NaN,0393925765,NaN,NaN,F.1946.h
2,1706700,"Hofhaimer, Paul",1459-1537,Harmoniae poeticae Pauli Hofheimeri : viri equ...,Harmoniae poeticae,"Harmoniae poeticae Pauli Hofheimeri, & Ludovic...","Horace ; Stomius, Johannes ; Senfl, Ludwig",1539.0,1539,Germany,...,NaN,RISM B/I 1539²⁶,"Part songs, Latin","5 parts, 8°",NaN,NaN,NaN,NaN,NaN,1070.c.12.(1.) ; 1213.i.1 ; G.727
3,1825532,NaN,NaN,Istituzioni e monumenti dell'arte musicale ita...,NaN,NaN,NaN,1931.0,1931-1939,Italy,...,v. 1-2. Andrea e Giovanni Gabrieli e la musica...,NaN,"Instrumental music ; Vocal music, Italian ; Sa...","6 volumes, illustrations, facsimiles (part col...",NaN,NaN,NaN,NaN,NaN,Hirsch IV.975 ; H.14
4,2225270,"Chapman, Mary, (Musician)",NaN,Eight ball studies suitable for use in girls' ...,NaN,NaN,NaN,1940.0,1940,England,...,NaN,NaN,Piano music ; Physical education and training,"1 score (12 pages ), 26 cm + 1 volume (8 pages...",NaN,NaN,NaN,NaN,NaN,D-07907.f.10
5,3050793,"Razzi, Serafino",1531-,"Santuario di laudi : o vero rime spirituali, p...",NaN,NaN,"Verdelot, Philippe",1609.0,1609,Italy,...,NaN,RISM B/I 1609⁸,"Hymns, Italian ; Propers (Music)","24, 240, 24 pages of music, 8°",NaN,NaN,NaN,NaN,NaN,11427.ee.25
6,3081869,Íslenzka stúdentafélag,NaN,Söngbók hins Íslenzka Stúdentafjelags,NaN,NaN,NaN,1894.0,1894,Iceland,...,Ættjarðarkvæði -- Alþýðuvísur -- Undir berum h...,NaN,Students' songs--Iceland ; Songs ; Music--Iceland,"103 pages, music, 15 x 18 cm, (oblong 8°)",NaN,NaN,NaN,NaN,NaN,866.k.15
7,3240772,"Sanchis Guarner, Manuel",NaN,Cancçoneret Valencià de Nadal : Segona edició ...,NaN,NaN,NaN,1973.0,1973,Spain,...,NaN,NaN,NaN,"55 pages, 24 cm",Els quaderns,no. 3 [Els quaderns],NaN,NaN,NaN,P.521/1605.(3.)
8,4083483,Liturgies.Latin Rite. Graduals. II. Local. Pas...,NaN,Graduale Pataviense (Wien 1511). Faksimile. He...,NaN,NaN,"Vaeterlein, Christian",1982.0,NaN,Germany,...,NaN,NaN,NaN,"vi, 294, 13 pages, music 25 x 33 cm",Das Erbe deutscher Musik. Bd.87. Abteilung Mit...,NaN,NaN,NaN,NaN,H.995.b./87
9,4162358,NaN,NaN,A B C D E F G H : The First Part of the Horn B...,NaN,First part of the horn book,NaN,1795.0,1795,England,...,NaN,NaN,NaN,folio,NaN,NaN,NaN,NaN,NaN,G.356.(6.)


In [27]:
#verificando o tamanho do dataset

musicas.shape

(1045507, 23)

In [28]:
# Alterando os nomes das colunas

colunas = {'BL record ID':'ID da gravação', 'Composer':'Compositor', 'Composer life dates':'Datas de vida do compositor', 'Title':'Titulo',
            'Standardised title':'Titulo padronizado', 'Other titles':'Outros titulos','Other names':'Outros nomes',
            'Publication date (standardised)':'Data de publicação (padronizada)', 'Publication date (not standardised)':'Data de publicação (não padronizada)',
             'Country of publication':'País de publicação', 'Place of publication':'Local de publicação', 'Publisher':'Editora','Notes':'Notas',
             'Contents':'Conteudos extras','Referenced in':'Referencias' ,'Subject/genre terms':'Genero','Physical description':'Descrição física',
             'Series title':'Titulo da série','Number within series':'Número dentro da série',
             'Publisher number':'Número da editora','BL shelfmark':'Marca de Prateleira'
            }

musicas.rename(columns=colunas, inplace=True)

In [29]:
musicas.head()

,ID da gravação,Compositor,Datas de vida do compositor,Titulo,Titulo padronizado,Outros titulos,Outros nomes,Data de publicação (padronizada),Data de publicação (não padronizada),País de publicação,...,Conteudos extras,Referencias,Genero,Descrição física,Titulo da série,Número dentro da série,ISBN,ISMN,Número da editora,Marca de Prateleira
0,1279866,NaN,NaN,The Penguin book of Canadian Folk songs,NaN,Canadian folk songs,"Fowke, Edith ; MacMillan, Keith (Keith Campbell)",1973.0,1973,England,...,A Fenian song -- Bold Wolfe -- The battle of t...,NaN,"Folk songs--Canada ; Folk songs, French--Canada","224 pages, music, 20 cm",NaN,NaN,0140708421,NaN,NaN,mH00/3305 ; X.439/3548
1,1312079,NaN,NaN,Anthology for The musician's guide to theory a...,NaN,"Musician's guide, anthology ; Musician's guide...","Clendinning, Jane Piper ; Marvin, Elizabeth West",2005.0,c2005,New York (State),...,NaN,NaN,Musical analysis--Music collections,"1 score (vii, 260 pages), 28 cm",NaN,NaN,0393925765,NaN,NaN,F.1946.h
2,1706700,"Hofhaimer, Paul",1459-1537,Harmoniae poeticae Pauli Hofheimeri : viri equ...,Harmoniae poeticae,"Harmoniae poeticae Pauli Hofheimeri, & Ludovic...","Horace ; Stomius, Johannes ; Senfl, Ludwig",1539.0,1539,Germany,...,NaN,RISM B/I 1539²⁶,"Part songs, Latin","5 parts, 8°",NaN,NaN,NaN,NaN,NaN,1070.c.12.(1.) ; 1213.i.1 ; G.727
3,1825532,NaN,NaN,Istituzioni e monumenti dell'arte musicale ita...,NaN,NaN,NaN,1931.0,1931-1939,Italy,...,v. 1-2. Andrea e Giovanni Gabrieli e la musica...,NaN,"Instrumental music ; Vocal music, Italian ; Sa...","6 volumes, illustrations, facsimiles (part col...",NaN,NaN,NaN,NaN,NaN,Hirsch IV.975 ; H.14
4,2225270,"Chapman, Mary, (Musician)",NaN,Eight ball studies suitable for use in girls' ...,NaN,NaN,NaN,1940.0,1940,England,...,NaN,NaN,Piano music ; Physical education and training,"1 score (12 pages ), 26 cm + 1 volume (8 pages...",NaN,NaN,NaN,NaN,NaN,D-07907.f.10


In [30]:
# Contagem/% de valores nulos no dataframe

frequencia = musicas.isna().sum() 
percentual_na = round((musicas.isna().sum() / len(musicas))*100, 2)
valores_na = pd.DataFrame([frequencia, percentual_na]).transpose().rename(columns={0:'Frequência', 1:'% de NAs'})
valores_na

,Frequência,% de NAs
ID da gravação,0.0,0.00
Compositor,42059.0,4.02
Datas de vida do compositor,583222.0,55.78
Titulo,22.0,0.00
Titulo padronizado,807217.0,77.21
Outros titulos,1033223.0,98.83
Outros nomes,794108.0,75.95
Data de publicação (padronizada),21494.0,2.06
Data de publicação (não padronizada),15216.0,1.46
País de publicação,334204.0,31.97


In [31]:
musicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045507 entries, 0 to 1045506
Data columns (total 23 columns):
 #   Column                                Non-Null Count    Dtype  
---  ------                                --------------    -----  
 0   ID da gravação                        1045507 non-null  int64  
 1   Compositor                            1003448 non-null  object 
 2   Datas de vida do compositor           462285 non-null   object 
 3   Titulo                                1045485 non-null  object 
 4   Titulo padronizado                    238290 non-null   object 
 5   Outros titulos                        12284 non-null    object 
 6   Outros nomes                          251399 non-null   object 
 7   Data de publicação (padronizada)      1024013 non-null  float64
 8   Data de publicação (não padronizada)  1030291 non-null  object 
 9   País de publicação                    711303 non-null   object 
 10  Local de publicação                   716348 non-null 

In [32]:
# Otimizando o tamanho da do dataframe

musicas['ID da gravação'] = musicas["ID da gravação"].astype('Int32')
musicas['Data de publicação (padronizada)'] = musicas["Data de publicação (padronizada)"].astype('Int16')

In [33]:
musicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045507 entries, 0 to 1045506
Data columns (total 23 columns):
 #   Column                                Non-Null Count    Dtype 
---  ------                                --------------    ----- 
 0   ID da gravação                        1045507 non-null  Int32 
 1   Compositor                            1003448 non-null  object
 2   Datas de vida do compositor           462285 non-null   object
 3   Titulo                                1045485 non-null  object
 4   Titulo padronizado                    238290 non-null   object
 5   Outros titulos                        12284 non-null    object
 6   Outros nomes                          251399 non-null   object
 7   Data de publicação (padronizada)      1024013 non-null  Int16 
 8   Data de publicação (não padronizada)  1030291 non-null  object
 9   País de publicação                    711303 non-null   object
 10  Local de publicação                   716348 non-null   object
 11

In [34]:
fim_pandas_musica = time.time()

---
### Trabalhando com o Dataset maior usando SPARK
---

In [35]:
inicio_spark_musica = time.time()

In [36]:
fim_spark_musica = time.time()

---
### Comparando os resultados
---

In [37]:

# Tempo de execução para o dataframe menor usando:
print('Tempo de execução para o dataframe menor usando:')

#Pandas
t1 = fim_pandas_netflix - inicio_pandas_netflix
print(f'Usando Pandas: {t1}')

#Spark
t2 = fim_spark_netflix - inicio_pandas_netflix
print(f'Usando Spark: {t2}')

# Tempo de execução para o dataframe menor usando:
print('Tempo de execução para o dataframe maior usando:')

#Pandas
t3 = fim_pandas_musica - inicio_pandas_netflix
print(f'Usando Pandas: {t3}')

#Spark
t4 = fim_spark_musica - inicio_spark_musica
print(f'Usando Spark: {t4}')



Tempo de execução para o dataframe menor usando:
Usando Pandas: 2.500178575515747
Usando Spark: 17.28699517250061
Tempo de execução para o dataframe maior usando:
Usando Pandas: 61.14903783798218
Usando Spark: 0.13293027877807617
